# Verificación y Exploración de Base de Datos CASEN 2022

Este notebook carga la base de datos STATA del CASEN 2022 y extrae/lista las variables por categorías temáticas.

## Objetivos:
- ✅ Cargar archivo .dta de CASEN 2022
- ✅ Verificar integridad de datos
- ✅ Listar todas las variables
- ✅ Organizar variables por categorías temáticas
- ✅ Mostrar estadísticas básicas por variable

In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import os
import sys
from pathlib import Path

# Agregar scripts al path
sys.path.insert(0, '../03_Scripts')

# Mostrar configuración
print("✅ Librerías importadas exitosamente")
print(f"📍 Directorio actual: {os.getcwd()}")
print(f"📍 Versión de Pandas: {pd.__version__}")
print(f"📍 Versión de NumPy: {np.__version__}")

✅ Librerías importadas exitosamente
📍 Directorio actual: /Users/estebanroman/Documents/GitHub/MuestreoCasen /Proyecto-Final-Muestreo/notebook_verificacion
📍 Versión de Pandas: 2.0.3
📍 Versión de NumPy: 1.24.3


## 1. Carga de Base de Datos CASEN 2022

In [2]:
# Ruta del archivo STATA
ruta_archivo = '../Base de datos Casen 2022 STATA_18 marzo 2024.dta'

# Verificar que el archivo existe
if os.path.exists(ruta_archivo):
    print(f"✅ Archivo encontrado: {os.path.basename(ruta_archivo)}")
    print(f"📊 Tamaño: {os.path.getsize(ruta_archivo) / 1024 / 1024:.2f} MB")
else:
    print(f"❌ No se encontró archivo en: {ruta_archivo}")
    print(f"\nArchivos disponibles en directorio padre:")
    parent_dir = os.path.dirname(ruta_archivo)
    for f in os.listdir(parent_dir):
        if f.endswith('.dta'):
            print(f"   - {f}")

✅ Archivo encontrado: Base de datos Casen 2022 STATA_18 marzo 2024.dta
📊 Tamaño: 1654.16 MB


In [ ]:
# Cargar la base de datos STATA
print("🔄 Cargando archivo STATA...")
print("⏳ Por favor espere, esto puede tomar algunos segundos...\n")

try:
    # Suprimir advertencia de Unicode/encoding
    # Stata puede tener caracteres con encoding diferente al UTF-8
    # La librería pyreadstat maneja automáticamente el fallback a latin-1
    import warnings
    
    with warnings.catch_warnings(record=True) as w:
        warnings.simplefilter("ignore", UnicodeWarning)
        df = pd.read_stata(ruta_archivo, convert_categoricals=False)
    
    # Limpiar nombres de columnas si hay espacios o caracteres especiales
    df.columns = df.columns.str.strip()
    
    print("✅ ¡Base de datos cargada exitosamente (con encoding correcto)!")
    print(f"\n📊 INFORMACIÓN DE LA BASE DE DATOS:")
    print(f"   • Observaciones (filas): {df.shape[0]:,}")
    print(f"   • Variables (columnas): {df.shape[1]}")
    print(f"   • Tamaño en memoria: {df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")
    print(f"\n📅 Primeras 5 observaciones:")
    df.head()
except Exception as e:
    print(f"❌ Error al cargar archivo: {e}")

🔄 Cargando archivo STATA...
⏳ Por favor espere, esto puede tomar algunos segundos...



## 2. Extracción y Categorización de Variables

Extraeremos todas las variables de la base de datos y las organizaremos por categorías temáticas.

In [ ]:
# Crear diccionario de categorización de variables basado en el nombre
# Esto es una clasificación inferida de CASEN

categorizacion = {
    'Identificación y Expansión': [
        'id_hogar', 'foliohogar', 'folio', 'ident', 'expr', 'expr_1', 
        'expansores', 'expd', 'region', 'comuna', 'zona', 'area'
    ],
    
    'Características del Hogar': [
        'numhogares', 'nmiembros', 'numper', 'v1', 'tipo_hogar',
        'thogar', 'pco', 'p01', 'p02', 'p03', 'p04', 'p05',
        'viviendasituaci', 'vivtipo', 'tipo_viv'
    ],
    
    'Jefe/a del Hogar': [
        'jefe', 'sexo_jefe', 'sexojf', 'edad_jefe', 'edadjf',
        'ecivil', 'eciv_jf', 'esc_jefe', 'escjf', 'esc',
        'oficio_jefe', 'oficojf', 'activ_jefe'
    ],
    
    'Ingresos y Salarios': [
        'ybase', 'ytrabm', 'yactiv', 'yotrm', 'yoprinc', 'ytotcorh',
        'ypc', 'ypchchogar', 'ymonedas', 'ingho', 'inghogar',
        'ingcorh', 'ingcormh', 'inghcorh', 'ytotal', 'ytotcor',
        'sueldo', 'salario', 'ingprincipal', 'ingotro'
    ],
    
    'Educación': [
        'p08', 'p09', 'p10', 'esc', 'escorig', 'escolari', 'escolaridad',
        'niveledu', 'educacion', 'p_ed', 'esc_jefe', 'escjf',
        'asiste', 'p15', 'p16', 'superior', 'educpersonas',
        'nivel', 'educa'
    ],
    
    'Empleo y Ocupación': [
        'activ', 'actividad', 'cotiza', 'ocup', 'ocupacion', 'oficio',
        'rama', 'rama_actividad', 'categor', 'categoria',
        'empresa', 'tamaño_empresa', 'tamano_empresa', 'empresa_p',
        'p19', 'p20', 'p21', 'p22', 'p23', 'p24', 'p25',
        'busco_trab', 'deseo_trab', 'razon_desemple', 'razon_inac',
        'ocupacion_principal', 'situacion_laboral'
    ],
    
    'Pobreza y Vulnerabilidad': [
        'pobreza', 'pobre', 'v16a', 'idpobre', 'ipobre',
        'incpobreza', 'linea_pobreza', 'lpobrezas',
        'lp', 'lpe', 'extremo', 'pobre_extremo'
    ],
    
    'Salud y Seguridad Social': [
        'prevision', 'stsalud', 'isapre', 'fonasa', 'mutual',
        'segurotrab', 'afiliacion', 'p31', 'p32', 'p33', 'p34',
        'salud', 'discapacidad', 'disc', 'accion', 'seguros'
    ],
    
    'Vivienda y Servicios Básicos': [
        'propiedad', 'prop', 'mutualista', 'arriendo', 'condicion_propiedad',
        'tenencia', 'material_piso', 'material_muros', 'material_techo',
        'agua', 'electricidad', 'gas', 'combustible', 'combustible_cocina',
        'servicios_basicos', 'sistemas_agua', 'alcantarillado', 'letrina',
        'v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10'
    ],
    
    'Características Demográficas': [
        'edad', 'sexo', 'estado_civil', 'ecivil', 'hijos',
        'numero_hijos', 'parentesco', 'parente', 'nacionalidad',
        'ind_migrat', 'migrat', 'tiempo_residencia', 'p11', 'p12', 'p13'
    ],
    
    'Bonos y Transferencias': [
        'bip', 'asignacion', 'bonoauto', 'bonofam', 'aps',
        'seguro_desemple', 'bono_marzo', 'bono', 'transferencias',
        'apoyo_social', 'beneficiarios'
    ],
    
    'Otras Variables': []  # Categoría por defecto
}

# Obtener todas las columnas del dataframe
todas_variables = df.columns.tolist()

# Crear un diccionario invertido para clasificar las variables
variables_clasificadas = {}
variables_sin_clasificar = []

for var in todas_variables:
    var_lower = var.lower()
    clasificada = False
    
    for categoria, palabras_clave in categorizacion.items():
        if categoria == 'Otras Variables':
            continue
            
        for palabra in palabras_clave:
            if palabra.lower() in var_lower:
                if categoria not in variables_clasificadas:
                    variables_clasificadas[categoria] = []
                variables_clasificadas[categoria].append(var)
                clasificada = True
                break
        
        if clasificada:
            break
    
    if not clasificada:
        variables_sin_clasificar.append(var)

# Agregar variables sin clasificar a "Otras Variables"
variables_clasificadas['Otras Variables'] = variables_sin_clasificar

print("✅ Variables extraídas y categorizadas")
print(f"\n📊 RESUMEN DE CATEGORIZACIÓN:")
print(f"   Total de variables: {len(todas_variables)}\n")

for categoria, variables in variables_clasificadas.items():
    print(f"   {categoria}: {len(variables)} variables")

## 3. Listado Detallado de Variables por Categoría

In [ ]:
# Crear tabla con información detallada de variables
print("\n" + "="*120)
print("LISTADO COMPLETO DE VARIABLES POR CATEGORÍA TEMÁTICA".center(120))
print("="*120 + "\n")

for categoria in sorted(variables_clasificadas.keys()):
    variables = sorted(variables_clasificadas[categoria])
    
    if len(variables) == 0:
        continue
    
    print(f"\n📁 {categoria.upper()}")
    print(f"{'─'*120}")
    print(f"   Total: {len(variables)} variables\n")
    
    # Mostrar variables en columnas
    for i, var in enumerate(variables, 1):
        # Obtener información de la variable
        dtype = str(df[var].dtype)
        n_valores = df[var].notna().sum()
        n_faltantes = df[var].isna().sum()
        
        print(f"   {i:2d}. {var:30s} | Tipo: {dtype:12s} | Válidos: {n_valores:8,d} | Faltantes: {n_faltantes:8,d}")

print("\n" + "="*120)

## 4. Tabla Resumida de Variables

In [ ]:
# Crear tabla resumen
resumen_categorias = []

for categoria in sorted(variables_clasificadas.keys()):
    variables = variables_clasificadas[categoria]
    
    if len(variables) == 0:
        continue
    
    # Calcular completitud promedio
    completitud = []
    for var in variables:
        pct_validos = (df[var].notna().sum() / len(df)) * 100
        completitud.append(pct_validos)
    
    completitud_promedio = np.mean(completitud) if completitud else 0
    
    resumen_categorias.append({
        'Categoría': categoria,
        'Cantidad de Variables': len(variables),
        'Completitud Promedio (%)': round(completitud_promedio, 1)
    })

resumen_df = pd.DataFrame(resumen_categorias)

print("\n📊 RESUMEN DE CATEGORÍAS:\n")
print(resumen_df.to_string(index=False))
print("\n" + "─"*80)
print(f"{'TOTAL':.<40} {resumen_df['Cantidad de Variables'].sum()} variables")
print(f"{'Completitud Promedio General':.<40} {resumen_df['Completitud Promedio (%)'].mean():.1f}%")

## 5. Estadísticas Descriptivas por Categoría (Muestreo)

In [ ]:
# Mostrar estadísticas descriptivas para variables numéricas clave por categoría
print("\n" + "="*100)
print("ESTADÍSTICAS DESCRIPTIVAS - VARIABLES NUMÉRICAS POR CATEGORÍA".center(100))
print("="*100 + "\n")

for categoria in sorted(variables_clasificadas.keys()):
    variables = variables_clasificadas[categoria]
    
    # Filtrar solo variables numéricas
    variables_numericas = [v for v in variables if df[v].dtype in ['int64', 'int32', 'float64', 'float32']]
    
    if len(variables_numericas) == 0:
        continue
    
    print(f"\n📊 {categoria.upper()}")
    print("─"*100)
    
    # Mostrar estadísticas de primeras 3 variables numéricas o todas si hay menos de 3
    vars_a_mostrar = variables_numericas[:min(3, len(variables_numericas))]
    
    for var in vars_a_mostrar:
        stats = df[var].describe()
        print(f"\n   {var}:")
        print(f"      Observaciones: {stats['count']:,.0f}")
        print(f"      Promedio: {stats['mean']:.2f}")
        print(f"      Desv. Est: {stats['std']:.2f}")
        print(f"      Mín: {stats['min']:.2f}")
        print(f"      Máx: {stats['max']:.2f}")
        print(f"      Mediana (50%): {stats['50%']:.2f}")

print("\n" + "="*100)

## 6. Exportar Variables a Archivo Excel

In [ ]:
# Crear archivo Excel con todas las variables organizadas por categoría
ruta_excel = '../05_Outputs/Variables_CASEN_2022_Categorizado.xlsx'

# Crear objeto ExcelWriter
with pd.ExcelWriter(ruta_excel, engine='openpyxl') as writer:
    
    # Primera hoja: Resumen
    resumen_df.to_excel(writer, sheet_name='Resumen', index=False)
    
    # Segunda hoja: Todas las variables con detalles
    todas_vars_detalles = []
    for categoria in sorted(variables_clasificadas.keys()):
        for var in sorted(variables_clasificadas[categoria]):
            todas_vars_detalles.append({
                'Categoría': categoria,
                'Variable': var,
                'Tipo de Dato': str(df[var].dtype),
                'Valores Válidos': df[var].notna().sum(),
                'Valores Faltantes': df[var].isna().sum(),
                'Completitud (%)': round((df[var].notna().sum() / len(df)) * 100, 2),
                'Valor Mínimo': df[var].min() if df[var].dtype in ['int64', 'float64'] else 'N/A',
                'Valor Máximo': df[var].max() if df[var].dtype in ['int64', 'float64'] else 'N/A',
                'Promedio': round(df[var].mean(), 2) if df[var].dtype in ['int64', 'float64'] else 'N/A'
            })
    
    detalle_df = pd.DataFrame(todas_vars_detalles)
    detalle_df.to_excel(writer, sheet_name='Detalle Completo', index=False)
    
    # Una hoja por cada categoría
    for categoria in sorted(variables_clasificadas.keys()):
        if len(variables_clasificadas[categoria]) == 0:
            continue
        
        cat_vars = []
        for var in sorted(variables_clasificadas[categoria]):
            cat_vars.append({
                'Variable': var,
                'Tipo': str(df[var].dtype),
                'Válidos': df[var].notna().sum(),
                'Faltantes': df[var].isna().sum(),
                'Completitud (%)': round((df[var].notna().sum() / len(df)) * 100, 2)
            })
        
        cat_df = pd.DataFrame(cat_vars)
        # Limitar nombre de hoja a 31 caracteres (límite de Excel)
        nombre_hoja = categoria[:31]
        cat_df.to_excel(writer, sheet_name=nombre_hoja, index=False)

print(f"✅ Archivo Excel creado exitosamente")
print(f"📁 Ubicación: {ruta_excel}")
print(f"   • Contiene {len(resumen_df)} hojas de resumen por categoría")
print(f"   • Total de variables exportadas: {len(detalle_df)}")